In [ ]:
using DrWatson
@quickactivate "masterarbeit"

In [ ]:
using CUDA
using BenchmarkTools
using ProgressMeter
using Flux
using LaTeXStrings
using Flux: train!
using GLMakie
using Printf
using Dates
using JLD2
using TOML
using StatsBase # for fit(histogram)

In [ ]:
Makie.inline!(true)
fontsize_theme = Theme(fontsize=35)
set_theme!(fontsize_theme)
wblue = Makie.wong_colors()[1]
worange = Makie.wong_colors()[2]
wgreen = Makie.wong_colors()[3]
wpink = Makie.wong_colors()[4]
wlblue = Makie.wong_colors()[5]
worange = Makie.wong_colors()[6]
wyellow = Makie.wong_colors()[7];

In [ ]:
using Revise

In [ ]:
using masterarbeit

In [ ]:
function jacobian2cl(m::Chain, cm::ChannelMapping, x::T) where {T <: AbstractArray{F}} where F<:Real
    cl1 = m[1]
    sl1 = m[2]
    cl2 = m[3]
    x2 = cl1(x)
    x2s = sl1(x2)
    det1 = abs.(cldet(cl1,  x[cl1.dimA+1:cl1.d,:], cl1.m( x[1:cl1.dimA,:])...))
    det2 = abs.(cldet(cl2, x2s[cl2.dimA+1:cl2.d,:], cl2.m(x2s[1:cl2.dimA,:])...))
    return abs(cmdet(cm)) .* det1 .* det2
end

In [ ]:
function lossf(m::Chain, cm::ChannelMapping, f::Function, x::T) where T<:AbstractArray{F} where F<:Real
    zi = cm(m(x))
    g = 1 ./ jacobian2cl(m, cm, x)
    fracs = abs.(f(zi) .- g) .^F(1.5) ./ g
    return sum(fracs) / size(x,2)
end

In [ ]:
f = single_gauss
ytozmap = HypercubeTocθωbar()
dim = 2
dimA = 1;

In [ ]:
batchsize = 16384
N_epochs = 30
bins = 10
learning_rate = 0.01
decay = 0.7
optimizer = Adam
;

In [ ]:
function subnet(dimA::Signed, dimB::Signed, bins::Signed, width=4)
    return Chain(
        Split(
            Chain(
                BatchNorm(dimA),
                Dense(dimA => width, relu),
                Dense(width => width, relu),
                Dense(width => dimB*(bins+1))  
                ), 
            Chain(
                BatchNorm(dimA),
                Dense(dimA => width, relu),
                Dense(width => width, relu),
                Dense(width => dimB*bins)
                )
            ) 
        ) |> gpu
end

In [ ]:
model = Flux.f32(Chain(
    CouplingLayer(dim, dimA, bins, subnet),
    masterarbeit.MaskLayer([false, true]),
    CouplingLayer(dim, dimA, bins, subnet),
) |> gpu);

In [ ]:
# first run to compile/test
xtest = CUDA.rand(dim,batchsize);

In [ ]:
model(xtest);

In [ ]:
f(ytozmap(xtest));

In [ ]:
lossf(model,ytozmap,f,xtest)

In [ ]:
Flux.withgradient(m-> lossf(m,ytozmap,f,xtest), model);

# Training

In [ ]:
losses = Float32[]

In [ ]:
current_learning_rate = learning_rate
for i in 1:3
    println("Training with learning rate  = $(current_learning_rate)")
    losses = train_NN(model, dim, lossf, losses, ytozmap, f, epochs=N_epochs, batchsize=batchsize, optimizer=optimizer, learning_rate=current_learning_rate)
    current_learning_rate = current_learning_rate * decay
end;

In [ ]:
losses[end]

In [ ]:
fig = Figure(size=(1500,1000))
ax = Axis(fig[1,1], xlabel="epoch", ylabel="loss", xlabelsize=50, ylabelsize=50)
lines!(1:length(losses), losses, linewidth=4, color=wblue, label="loss")
n = 10
lines!(n:length(losses), moving_average(losses, n), linewidth=4, color=worange, label="$n epoch \n moving average")
fig[1,2] = Legend(fig, ax)
save("singlegauss_nis_loss.png", fig)
fig

In [ ]:
samples = sample_NN(model, ytozmap, dim, 10^7, batchsize);

In [ ]:
xticks = [-1.0, -0.5, 0.0, 0.5, 1.0]
yticks = [0.0, 0.25, 0.5, 0.75, 1.0];

In [ ]:
histo = fit(Histogram, (samples[1,:], samples[2,:]), nbins=100)
histo_n = StatsBase.normalize(histo, mode=:pdf)
fig = Figure(size=(1200,1000), figure_padding=40)
ax = Axis(fig[1,1], xlabel=L"x_1", ylabel=L"x_2", xticks=xticks, yticks=yticks, aspect=1, xlabelsize=50, ylabelsize=50)
hm = heatmap!(histo.edges[1], histo.edges[2], histo_n.weights, colorrange=(0,1))
fig[1, 2] = GridLayout(width = 20)
Colorbar(fig[1,3], hm, width=40)
save("singlegauss_nis_samples.png", fig)
fig

In [ ]:
fig = Figure(size=(1200,1000), figure_padding=40)
ax = Axis(fig[1,1], xlabel=L"x_1", ylabel=L"x_2", xticks=xticks, yticks=yticks, aspect=1, xlabelsize=50, ylabelsize=50)
xs = LinRange(-1.0, 1.0, 1001)
ys = LinRange(0.0, 1.0, 1001)
zs = [f([x;y])[1] for x in xs, y in ys] ./ 1.11549
heatmap!(xs, ys, zs)
#fig[1, 2] = GridLayout(width = 20)
#Colorbar(fig[1,3], hm, width=40)
save("singlegauss_truth.png", fig)
fig

In [ ]:
wi_vegas = load_object("vegas_weights.jld2");

In [ ]:
function weights2cl(m::Chain, cm::ChannelMapping, f::Function, x::T) where {T <: AbstractArray}
    return jacobian2cl(m, cm, x) .* f(cm(m(x)))
end

function weights2cl_chunked(m, dim, cm, f, N, batchsize)
    if (N%batchsize != 0) 
        x = CUDA.rand(dim, N%batchsize)
        weights = weights2cl(m, cm, f, x)
        inputs = x
        runs = N ÷ batchsize 
    else
        x = CUDA.rand(dim,   batchsize)
        weights = weights2cl(m, cm, f, x)
        inputs = x
        runs = N ÷ batchsize - 1
    end
    for i in 1:runs
        x = CUDA.rand(dim, batchsize)
        weights = hcat(weights, weights2cl(m, cm, f, x))
        inputs = hcat(inputs, x)
    end
    return weights |> cpu, inputs
end

wi_m, x_for_wi = weights2cl_chunked(model, dim, ytozmap, f,  10^7, batchsize)
wi = wi_m[1,:];

In [ ]:
f_over_g = wi
mcint_nis = sum(f_over_g) / size(samples,2)
mcerror = sqrt(sum((f_over_g  .- mcint_nis).^2) / (size(samples,2)-1))
println("mc integral = $mcint_nis")
println("standard deviation = $mcerror")

In [ ]:
mcint = 1.11549

In [ ]:
w_n = wi ./ mcint
w_avg = mean(w_n)
w_max = maximum(w_n)
efficiency = w_avg / w_max
println("mean weight = $w_avg")
println("max weight = $w_max")
println("unweighting efficiency = $efficiency")

In [ ]:
fig = Figure(size=(1500,1000))
ax = Axis(fig[1,1], ylabel=L"N", xlabel=L"w", yscale=Makie.pseudolog10, yticks=[0, 10^1, 10^3, 10^5, 10^7], xticks=[0.5,1.0,1.5,2.0], xlabelsize=50, ylabelsize=50)
stephist!(wi_vegas, color=wblue, linewidth=3, label="VEGAS", bins=50) 
stephist!(w_n, color=worange, linewidth=3, label="NIS", bins=50) 
xlims!(0.5,2.0)
fig[1,2] = Legend(fig, ax)
save("singlegauss_weights.png", fig)
fig

In [ ]:
fig = Figure(size=(1500,1000))
ax = Axis(fig[1,1], xlabel=L"x_1", ylabel="probability density", aspect=1, xlabelsize=50)
xs = LinRange(-1.0, 1.0, 1001)
lines!(xs, [f([x])[1] for x in xs] ./ 1.49365, color=worange, linewidth=3, label="ground truth")
stephist!(samples[1,:], bins=40, normalization=:pdf, color=wblue, linewidth=3, label="sample distribution")
fig[1,2] = Legend(fig, ax)

save("singlegauss_nis_compare.png", fig)
fig